In [1]:
import torch
from generator import RandomGraphDataset
from network import Network

2024-03-20 12:47:04.352110: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native
2024-03-20 12:47:04.971686: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native
2024-03-20 12:47:04.971744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native
2024-03-20 12:47:04.971749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If 

In [ ]:
def train(model, train_dataset, validation_dataset=None, optimizer=None, epochs=10, batch_size=5):
    x_loss_weight = 0.5
    h_loss_weight = 1 - x_loss_weight

    loss_edges_train, loss_reach_train, loss_parents_train = [], [], []
    loss_edges_val, loss_reach_val, loss_parents_val = [], [], []

    for epoch in range(epochs):
        batch_count = len(train_dataset) // batch_size
        
        cumulated_loss_edges_epoch, cumulated_loss_reach_epoch, cumulated_loss_parents_epoch = 0, 0, 0

        for i in range(batch_count):
            model.train()
            cumulated_loss_edges, cumulated_loss_reach, cumulated_loss_parents = 0, 0, 0
            for j in range(i*batch_size, (i+1)*batch_size):
                graph = train_dataset[j] 
                loss_edges, loss_reach, loss_parents = model(graph)
                loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1] # loss for the edges

                cumulated_loss_edges += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
                cumulated_loss_reach += loss_reach
                cumulated_loss_parents += loss_parents

            cumulated_loss_edges /= batch_size
            cumulated_loss_reach /= batch_size
            cumulated_loss_parents /= batch_size
            
            optimizer.zero_grad()
            cumulated_loss_edges.backward()
            optimizer.step()

            cumulated_loss_edges_epoch += cumulated_loss_edges
            cumulated_loss_reach_epoch += cumulated_loss_reach
            cumulated_loss_parents_epoch += cumulated_loss_parents

        # Convert tensors to lists and append to the respective lists
        loss_edges_train.append(cumulated_loss_edges_epoch.item() / batch_count)
        loss_reach_train.append(cumulated_loss_reach_epoch.item() / batch_count)
        loss_parents_train.append(cumulated_loss_parents_epoch.item() / batch_count)

        if validation_dataset:
            model.eval()
            with torch.no_grad():
                cumulated_loss_edges_val, cumulated_loss_reach_val, cumulated_loss_parents_val = 0, 0, 0
                for k in range(len(validation_dataset)):
                    graph = validation_dataset[k]
                    loss_edges, loss_reach, loss_parents = model(graph)
                    loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1] # loss for the edges

                    cumulated_loss_edges_val += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
                    cumulated_loss_reach_val += loss_reach
                    cumulated_loss_parents_val += loss_parents 

                cumulated_loss_edges_val /= len(validation_dataset)
                cumulated_loss_reach_val /= len(validation_dataset)
                cumulated_loss_parents_val /= len(validation_dataset)

                loss_edges_val.append(cumulated_loss_edges_val.item())
                loss_reach_val.append(cumulated_loss_reach_val.item())
                loss_parents_val.append(cumulated_loss_parents_val.item())

                print(f'Epoch {epoch}, loss_edges {cumulated_loss_edges_epoch.item() / batch_count}, loss_reach {cumulated_loss_reach_epoch.item() / batch_count}, loss_parents {cumulated_loss_parents_epoch.item() / batch_count}, loss_edges_val {cumulated_loss_edges_val.item()}, loss_reach_val {cumulated_loss_reach_val.item()}, loss_parents_val {cumulated_loss_parents_val.item()}')
        
        else:
            print(f'Epoch {epoch}, loss_edges {cumulated_loss_edges_epoch.item() / batch_count}, loss_reach {cumulated_loss_reach_epoch.item() / batch_count}, loss_parents {cumulated_loss_parents_epoch.item() / batch_count}')

    if validation_dataset:
        return loss_edges_train, loss_reach_train, loss_parents_train, loss_edges_val, loss_reach_val, loss_parents_val
    return loss_edges_train, loss_reach_train, loss_parents_train

In [ ]:
from torch.utils.data import random_split
n=[20, 100]
p=0.3
dataset = RandomGraphDataset(root='./data/medium', gen_num_graph=400, n=n, p=p)
train_dataset, test_dataset = random_split(dataset, [350, 50])

In [ ]:
optimizer = torch.optim.Adam
model = Network()
lr = 0.0002

loss_edges_train, loss_reach_train, loss_parents_train = train(model=model, train_dataset=train_dataset,
      optimizer=optimizer(model.parameters(), lr=lr), epochs=80, batch_size=32)

In [ ]:
optimizer = torch.optim.Adam
model = Network()
lr = 0.0002

loss_edges_train, loss_reach_train, loss_parents_train, loss_edges_val, loss_reach_val, loss_parents_val = train(model=model, train_dataset=train_dataset, validation_dataset=test_dataset,
      optimizer=optimizer(model.parameters(), lr=lr), epochs=120, batch_size=32)

In [ ]:
torch.save(model.state_dict(), 'model_0002_350_120.pth')

In [ ]:
from generator import RandomGraphDataset

In [ ]:
# split the dataset into training and testing using the train_test_split function
from torch.utils.data import random_split
n=[200, 500]
p=0.3
dataset_test = RandomGraphDataset(root='./data/OOD/200', gen_num_graph=34, n=n, p=p)

In [ ]:
x_loss_weight = h_loss_weight = 0.5
model.eval() 

with torch.no_grad():
    cumulated_loss_edges_test, cumulated_loss_reach_test, cumulated_loss_parents_test = 0, 0, 0
    for graph in dataset_test:
        loss_edges, loss_reach, loss_parents = model(graph)
        loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1]  # loss for the edges

        cumulated_loss_edges_test += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
        cumulated_loss_reach_test += loss_reach
        cumulated_loss_parents_test += loss_parents

    cumulated_loss_edges_test /= len(dataset_test)
    cumulated_loss_reach_test /= len(dataset_test)
    cumulated_loss_parents_test /= len(dataset_test)

In [ ]:
cumulated_loss_edges_test, cumulated_loss_reach_test, cumulated_loss_parents_test